In [55]:
# riss.kr 에서 특정 키워드로 논문 / 학술 자료 검색하기

#Step 1. 필요한 모듈을 로딩합니다
from selenium import webdriver
import time 

#Step 2. 사용자에게 검색 관련 정보들을 입력 받습니다.
print("=" *100)
print(" 이 크롤러는 RISS 사이트의 논문 및 학술자료 수집용 웹크롤러입니다.")
print("=" *100)
query_txt = input('1.수집할 자료의 키워드는 무엇입니까?(여러개일 경우 , 로 구분하여 입력): ')

print('위 키워드로 아래의 장르 중 어떤 장르의 정보를 수집할까요?')
print('1.학위논문\t2.국내학술논문\t3.해외학술논문\t4.학술자\n5.단행본\t6.공개강의\t7.연구보고서\n')
keyword = int(input('위 장르 중 수집할 장르의 번호를 입력하세요: '))

#Step 3. 수집된 데이터를 저장할 파일 이름 입력받기 
# ft_name = input('2.결과를 저장할 txt형식의 파일명을 쓰세요(예: c:\\temp\\riss.txt): ')
fc_name = input('3.결과를 저장할 csv형식의 파일명을 쓰세요(예: c:\\temp\\riss.csv): ')
fx_name = input('4.결과를 저장할 xls형식의 파일명을 쓰세요(예: c:\\temp\\riss.xls): ')

#Step 4. 크롬 드라이버 설정 및 웹 페이지 열기
chrome_path = "c:/temp/chromedriver_85/chromedriver.exe"
driver = webdriver.Chrome(chrome_path)

url = 'http://www.riss.kr/'
driver.get(url)
time.sleep(2)
driver.maximize_window() # 페이지 최대화 하기

#Step 5. 자동으로 검색어 입력 후 조회하기
element = driver.find_element_by_id("query")
driver.find_element_by_id("query").click( )
element.send_keys(query_txt)
element.send_keys("\n")
time.sleep(2)

# 6. 입력한 키워드로 들어가기
if keyword == 1:
    driver.find_element_by_link_text('학위논문').click()
    time.sleep(2)
elif keyword == 2:
    driver.find_element_by_link_text('국내학술논문').click()
    time.sleep(2)
elif keyword == 3:
    driver.find_element_by_link_text('해외학술논문').click()
    time.sleep(2)
elif keyword == 4:
    driver.find_element_by_link_text('학술지').click()
    time.sleep(2)
elif keyword == 5:
    driver.find_element_by_link_text('단평본').click()
    time.sleep(2)
if keyword == 6:
    driver.find_element_by_link_text('공개강의').click()
    time.sleep(2)
if keyword == 7:
    driver.find_element_by_link_text('연구보고서').click()
    time.sleep(2)
    
#Step 7.Beautiful Soup 로 본문 내용만 추출하기
from bs4 import BeautifulSoup
html_1 = driver.page_source
soup_1 = BeautifulSoup(html_1, 'html.parser')

content_1 = soup_1.find('div','srchResultListW').find_all('li')
# for i in content_1 :
#     print(i.get_text().replace("\n",""))

#Step 8. 총 검색 건수를 보여주고 수집할 건수 입력받기
import math
total_cnt = soup_1.find('div','searchBox').find('span','num').get_text()
print('검색하신 키워드 %s (으)로 총 %s 건의 학위논문이 검색되었습니다' %(query_txt,total_cnt))
collect_cnt = int(input('이 중에서 몇 건을 수집하시겠습니까?: '))
collect_page_cnt = math.ceil(collect_cnt / 10)
print('%s 건의 데이터를 수집하기 위해 %s 페이지의 게시물을 조회합니다.' %(collect_cnt,collect_page_cnt))

#Step 9. 각 항목별로 데이터를 추출하여 리스트에 저장하기
no2 = [ ]        #번호 저장
title2 = [ ]     #논문제목 저장
writer2 = [ ]    #논문저자 저장
org2 = [ ]       #소속기관 저장
no = 1
year2 = [] # 발행년도 저장
books = [] # 논문집/자료집
url2 = [] # 논문 URL 저장

# 다음 페이지 번호 만들기
page_no=[ ]

for i in range(10,collect_cnt) :
    if i % 10 == 0 :
            page_no.append(i + 1)
            
c = 0

for a in range(1, collect_page_cnt + 1) :
    
    html_2 = driver.page_source
    soup_2 = BeautifulSoup(html_2, 'html.parser')

    content_2 = soup_2.find('div','srchResultListW').find_all('li')
    
    for b in content_2 :    
        #1. 논문제목 있을 경우만
        try :
            title = b.find('div','cont').find('p','title').get_text()
        except :
            continue
        else :
            print('1.번호:',no)
            no2.append(no)

            print('2.논문제목:',title)
            title2.append(title)
            
            writer = b.find('span','writer').get_text()
            print('3.저자:',writer)
            writer2.append(writer)

            org = b.find('span','assigned').get_text()
            print('4.소속기관:' , org)
            org2.append(org)
            
            year = b.find('p','etc').find_all('span')[2].get_text()
            print('5.발표년도:' , year)
            year2.append(year)
            
            book = b.find('div', 'cont').find('p', 'etc').find_all('span')
            print('6.논문집/자료집:' , book[3].get_text())
            books.append(book[3].get_text())
            
            url = b.find('div', 'cont').find('p', 'title').a['href']
            print('7.논문 URL:' , 'http://www.riss.kr'+url)
            url2.append('http://www.riss.kr' + url)
            
            no += 1
            print("\n")
            
            if no > collect_cnt :
                break

    time.sleep(1)        # 페이지 변경 전 1초 대기 

    a += 1

    c = math.floor(a/10)-1

    if no > collect_cnt:
        break
    elif a == page_no[c]:
        driver.find_element_by_link_text('다음 페이지로').click()
    else :
        driver.find_element_by_link_text('%s' %a).click() # 다음 페이지번호 클릭 


print("요청하신 작업이 모두 완료되었습니다")
    

# Step 10. 수집된 데이터를 xls와 csv 형태로 저장하기
import pandas as pd 

df = pd.DataFrame()
df['번호']=no2
df['제목']=pd.Series(title2)
df['저자']=pd.Series(writer2)
df['소속(발행)기관']=pd.Series(org2)
df['발행년도']=pd.Series(year2)
df['논문집/자료집']=pd.Series(books)
df['논문 URL'] = pd.Series(url2)

# xls 형태로 저장하기
df.to_excel(fx_name,index=False, encoding="utf-8")

# csv 형태로 저장하기
df.to_csv(fc_name,index=False, encoding="utf-8-sig")

print('요청하신 데이터 수집 작업이 정상적으로 완료되었습니다')

 이 크롤러는 RISS 사이트의 논문 및 학술자료 수집용 웹크롤러입니다.
1.수집할 자료의 키워드는 무엇입니까?(여러개일 경우 , 로 구분하여 입력): 해양자원
위 키워드로 아래의 장르 중 어떤 장르의 정보를 수집할까요?
1.학위논문	2.국내학술논문	3.해외학술논문	4.학술자
5.단행본	6.공개강의	7.연구보고서

위 장르 중 수집할 장르의 번호를 입력하세요: 2
3.결과를 저장할 csv형식의 파일명을 쓰세요(예: c:\temp\riss.csv): c:\temp\riss.csv
4.결과를 저장할 xls형식의 파일명을 쓰세요(예: c:\temp\riss.xls): c:\temp\riss.xls
검색하신 키워드 해양자원 (으)로 총 3,586 건의 학위논문이 검색되었습니다
이 중에서 몇 건을 수집하시겠습니까?: 20
20 건의 데이터를 수집하기 위해 2 페이지의 게시물을 조회합니다.
1.번호: 1
2.논문제목: 발표논문 / 해양생물자원으로서 해조류 : 생물활성물질의 정제와 분자적 응용
3.저자: 홍용기(Yong Ki Hong)
4.소속기관: 한국조류학회
5.발표년도: 2000
6.논문집/자료집: 국제심포지움 일정 및 발표논문집 - 21세기, 해양환경과 해양생물자원의 전망
7.논문 URL: http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=508c47ca1906d5ecffe0bdc3ef48d419


1.번호: 2
2.논문제목: 수거된 해양폐기물 자원화 기술 개발(Ⅰ)
3.저자: 길상인(Sang-In Keel), 윤진한(Jin-Han Yun), 최연석(Yeon-Seok Choi), 강창구(Chang-Gu Kang), 유정석(Jeong-Seok Yu)
4.소속기관: 한국해양환경·에너지학회
5.발표년도: 2002
6.논문집/자료집: 한국해양환경·에너지학회지
7.논문 URL: http://www.riss.kr/search/detail/DetailView.do?p_mat

<ipython-input-55-f94ca76d8b8c>:173: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  df.to_excel(fx_name,index=False, encoding="utf-8")
